In [1]:
import collections

import anndata
import pandas as pd
import scipy.sparse as sp_sparse
import tables

h5_file = "../input/NSCLC_GSE127471_expression.h5"
# 메타데이터 불러오기
metadata = pd.read_csv("../input/NSCLC_GSE127471_CellMetainfo_table.tsv", sep="\t", index_col=0)

CountMatrix = collections.namedtuple("CountMatrix", ["feature_ref", "barcodes", "matrix"])

def get_matrix_from_h5(filename):
    with tables.open_file(filename, "r") as f:
        mat_group = f.get_node(f.root, "matrix")
        barcodes = f.get_node(mat_group, "barcodes").read()
        data = getattr(mat_group, "data").read()
        indices = getattr(mat_group, "indices").read()
        indptr = getattr(mat_group, "indptr").read()
        shape = getattr(mat_group, "shape").read()
        matrix = sp_sparse.csc_matrix((data, indices, indptr), shape=shape)

        feature_ref = {}
        feature_group = f.get_node(mat_group, "features")
        feature_ids = getattr(feature_group, "id").read()
        feature_names = getattr(feature_group, "name").read()
        feature_types = getattr(feature_group, "feature_type").read()
        feature_ref["id"] = feature_ids
        feature_ref["name"] = feature_names
        feature_ref["feature_type"] = feature_types
        tag_keys = getattr(feature_group, "_all_tag_keys").read()
        for key in tag_keys:
            key = key.decode("utf-8")
            feature_ref[key] = getattr(feature_group, key).read()

        return CountMatrix(feature_ref, barcodes, matrix)


mtx = get_matrix_from_h5(h5_file)
adata = anndata.AnnData(X=mtx.matrix.transpose())
# AnnData 객체에 메타데이터 및 변수 이름 설정
adata.obs = metadata
adata.obs.index = mtx.barcodes.astype(str)
adata.var.index = mtx.feature_ref["name"]
adata.var["gene_ids"] = mtx.feature_ref["id"]
adata

In [2]:
type(filtered_feature_bc_matrix)

__main__.CountMatrix

In [28]:
import anndata

# from scipy.io import mmread
import pandas as pd

# mtx 희소 행렬 불러오기
# X = mmread(filtered_feature_bc_matrix.matrix)

# AnnData 객체 생성
# adata = anndata.AnnData(X=X.transpose().tocsr())
adata = anndata.AnnData(X=filtered_feature_bc_matrix.matrix.transpose())

# 세포 메타데이터 불러오기
cell_meta = pd.read_csv("../input/NSCLC_GSE127471_CellMetainfo_table.tsv", sep="\t", index_col=0)

# AnnData 객체에 메타데이터 및 변수 이름 설정
adata.obs = cell_meta
adata.obs.index = filtered_feature_bc_matrix.barcodes.astype(str)
adata.var.index = filtered_feature_bc_matrix.feature_ref["name"]
adata.var["gene_ids"] = filtered_feature_bc_matrix.feature_ref["id"]

In [27]:
type(filtered_feature_bc_matrix.barcodes.astype(str))

numpy.ndarray

In [29]:
adata.obs.head()

,UMAP_1,UMAP_2,Celltype (malignancy),Celltype (major-lineage),Celltype (minor-lineage),Cluster,Patient,Sample,Source,Stage
AACGTTGGTCATACTG-1,-1.411341,6.299751,Immune cells,B,B,7,P1,P1,PBMC,Primary
AAGACCTCACAGGCCT-1,-1.240770,6.127520,Immune cells,B,B,7,P1,P1,PBMC,Primary
ACCGTAAGTCTTGTCC-1,-1.207882,6.265944,Immune cells,B,B,7,P1,P1,PBMC,Primary
ACGCAGCGTGTGCCTG-1,-1.991852,6.321991,Immune cells,B,B,7,P1,P1,PBMC,Primary
ACGGAGACAAGCCTAT-1,-2.926991,6.180597,Immune cells,B,B,7,P1,P1,PBMC,Primary


In [30]:
import collections
import anndata
import pandas as pd
import scipy.sparse as sp_sparse
import tables

def get_matrix_from_h5(filename):
    """
    HDF5 파일에서 카운트 매트릭스를 로드합니다.

    Parameters:
    filename (str): HDF5 파일의 경로.

    Returns:
    CountMatrix: 특징 참조, 바코드 및 매트릭스를 포함하는 Named 튜플.
    """
    with tables.open_file(filename, "r") as f:
        CountMatrix = collections.namedtuple("CountMatrix", ["feature_ref", "barcodes", "matrix"])
        mat_group = f.get_node(f.root, "matrix")
        barcodes = f.get_node(mat_group, "barcodes").read()
        data = getattr(mat_group, "data").read()
        indices = getattr(mat_group, "indices").read()
        indptr = getattr(mat_group, "indptr").read()
        shape = getattr(mat_group, "shape").read()
        matrix = sp_sparse.csc_matrix((data, indices, indptr), shape=shape)

        feature_ref = {}
        feature_group = f.get_node(mat_group, "features")
        feature_ids = getattr(feature_group, "id").read()
        feature_names = getattr(feature_group, "name").read()
        feature_types = getattr(feature_group, "feature_type").read()
        feature_ref["id"] = feature_ids
        feature_ref["name"] = feature_names
        feature_ref["feature_type"] = feature_types
        tag_keys = getattr(feature_group, "_all_tag_keys").read()
        for key in tag_keys:
            key = key.decode("utf-8")
            feature_ref[key] = getattr(feature_group, key).read()

        return CountMatrix(feature_ref, barcodes, matrix)

def make_anndata(input_path: str, h5_file:str, tsv_file:str):
    # HDF5 파일 경로
    h5_file = input_path + h5_file
    # 메타데이터 로드
    metadata = pd.read_csv(input_path + tsv_file, sep="\t", index_col=0)
    # HDF5 파일에서 카운트 매트릭스 로드
    mtx = get_matrix_from_h5(h5_file)
    # AnnData 객체 생성
    adata = anndata.AnnData(X=mtx.matrix.transpose())
    # AnnData 객체에 메타데이터 및 변수 이름 설정
    adata.obs = metadata
    adata.obs.index = mtx.barcodes.astype(str)
    adata.var.index = mtx.feature_ref["name"]
    adata.var["gene_ids"] = mtx.feature_ref["id"]
    return adata

adata = make_anndata("../input/", "NSCLC_GSE127471_expression.h5", "NSCLC_GSE127471_CellMetainfo_table.tsv")

In [31]:
adata

AnnData object with n_obs × n_vars = 1108 × 10700
    obs: 'UMAP_1', 'UMAP_2', 'Celltype (malignancy)', 'Celltype (major-lineage)', 'Celltype (minor-lineage)', 'Cluster', 'Patient', 'Sample', 'Source', 'Stage'
    var: 'gene_ids'

In [35]:
import anndata
import pandas as pd
import scipy.sparse as sp_sparse
import tables

def get_matrix_from_h5(filename:str):
    """
    HDF5 파일에서 카운트 매트릭스를 로드합니다.

    Parameters:
    filename (str): HDF5 파일의 경로.

    Returns:
    tuple: 특징 참조, 바코드 및 매트릭스를 담은 튜플.
    """
    with tables.open_file(filename, "r") as f:
        mat_group = f.get_node(f.root, "matrix")
        barcodes = f.get_node(mat_group, "barcodes").read()
        data = getattr(mat_group, "data").read()
        indices = getattr(mat_group, "indices").read()
        indptr = getattr(mat_group, "indptr").read()
        shape = getattr(mat_group, "shape").read()
        matrix = sp_sparse.csc_matrix((data, indices, indptr), shape=shape)

        feature_ref = {}
        feature_group = f.get_node(mat_group, "features")
        feature_ids = getattr(feature_group, "id").read()
        feature_names = getattr(feature_group, "name").read()
        feature_types = getattr(feature_group, "feature_type").read()
        feature_ref["id"] = feature_ids
        feature_ref["name"] = feature_names
        feature_ref["feature_type"] = feature_types
        tag_keys = getattr(feature_group, "_all_tag_keys").read()
        for key in tag_keys:
            key = key.decode("utf-8")
            feature_ref[key] = getattr(feature_group, key).read()

        return feature_ref, barcodes, matrix

def make_anndata(input_path: str, h5_file: str, tsv_file: str):
    """
    HDF5 파일로부터 카운트 매트릭스를 로드하여 AnnData 객체를 생성합니다.

    Parameters:
    input_path (str): 입력 파일의 경로.
    h5_file (str): HDF5 파일의 이름.
    tsv_file (str): TSV 파일의 이름.

    Returns:
    anndata.AnnData: AnnData 객체.
    """
    # HDF5 파일 경로
    h5_file = input_path + h5_file
    # 메타데이터 로드
    metadata = pd.read_csv(input_path + tsv_file, sep="\t", index_col=0)
    # HDF5 파일에서 카운트 매트릭스 로드
    feature_ref, barcodes, matrix = get_matrix_from_h5(h5_file)
    # AnnData 객체 생성
    adata = anndata.AnnData(X=matrix.transpose())
    # AnnData 객체에 메타데이터 및 변수 이름 설정
    adata.obs = metadata
    adata.obs.index = barcodes.astype(str)
    adata.var.index = feature_ref["name"]
    adata.var["gene_ids"] = feature_ref["id"]
    return adata


for i in ["NSCLC_GSE127471"]:
    adata = make_anndata("../input/", f"{i}_expression.h5", f"{i}_CellMetainfo_table.tsv")
    adata.write(f"../output/{i}.h5ad", compression="gzip")
